In [1]:
# 📦 Step 1: Import Required Libraries
import pandas as pd
import numpy as np
from datetime import timedelta
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')



In [2]:
# 🗃️ Step 2: Load the Dataset
df = pd.read_csv("online_retail.csv", encoding='ISO-8859-1')

df = df.dropna(subset=['Customer ID'])



In [3]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15Cm Christmas Glass Ball 20 Lights,12,01-12-2009 07:45,6.95,13085,United Kingdom
1,489434,79323P,Pink Cherry Lights,12,01-12-2009 07:45,6.75,13085,United Kingdom
2,489434,79323W,White Cherry Lights,12,01-12-2009 07:45,6.75,13085,United Kingdom
3,489434,22041,Record Frame 7 Single Size,48,01-12-2009 07:45,2.10,13085,United Kingdom
4,489434,21232,Strawberry Ceramic Trinket Box,24,01-12-2009 07:45,1.25,13085,United Kingdom


In [4]:
# 🧹 Step 3: Preprocessing
df['CustomerID'] = df['Customer ID'].astype(str)
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce', dayfirst=True)
df['TotalPrice'] = df['Quantity'] * df['Price']
df['IsCancelled'] = df['Invoice'].astype(str).str.startswith('C').astype(int)



In [5]:
# 📅 Step 4: Set Reference Date
snapshot_date = df['InvoiceDate'].max() + timedelta(days=1)



In [6]:
# 🎯 Step 5: Group and Feature Extraction
customer_gp = df.groupby('CustomerID')

features = pd.DataFrame()
features['Recency'] = customer_gp['InvoiceDate'].max().apply(lambda x: (snapshot_date - x).days)
features['FirstPurchaseDaysAgo'] = customer_gp['InvoiceDate'].min().apply(lambda x: (snapshot_date - x).days)
features['Frequency'] = customer_gp['Invoice'].nunique()
features['TotalQuantity'] = customer_gp['Quantity'].sum()
features['Monetary'] = customer_gp['TotalPrice'].sum()
features['AvgOrderValue'] = features['Monetary'] / features['Frequency']

# 🧾 Return related
features['TotalReturns'] = customer_gp['IsCancelled'].sum()
features['ReturnRate'] = features['TotalReturns'] / features['Frequency']

# 🧮 Quantity & Price Stats
features['AvgQuantityPerInvoice'] = customer_gp['Quantity'].mean()
features['MaxQuantity'] = customer_gp['Quantity'].max()
features['MinQuantity'] = customer_gp['Quantity'].min()
features['StdQuantity'] = customer_gp['Quantity'].std()

features['MeanPrice'] = customer_gp['Price'].mean()
features['MaxPrice'] = customer_gp['Price'].max()
features['MinPrice'] = customer_gp['Price'].min()
features['StdPrice'] = customer_gp['Price'].std()

# 🕒 Time Features
df['Hour'] = df['InvoiceDate'].dt.hour
df['Weekday'] = df['InvoiceDate'].dt.dayofweek
features['MostCommonHour'] = customer_gp['Hour'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
features['MostCommonWeekday'] = customer_gp['Weekday'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)

# 🕐 Days Between Orders
def avg_days_between(dates):
    dates = sorted(dates)
    if len(dates) < 2:
        return np.nan
    diffs = np.diff(dates)
    return np.mean(diffs).days

features['AvgDaysBetweenOrders'] = customer_gp['InvoiceDate'].agg(lambda x: avg_days_between(x.tolist()))

# 🌍 Country (optional)
# features['Country'] = customer_gp['Country'].first()  # uncomment for label encoding later





In [8]:
#  Fill Missing
features.fillna(0, inplace=True)

#  Normalize for modeling (optional)
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

#  Save
features.to_csv("full_customer_features.csv", index=True)
print("Extracted all features and saved to full_customer_features.csv")

#  Show preview
features.head()

Extracted all features and saved to full_customer_features.csv


,Recency,FirstPurchaseDaysAgo,Frequency,TotalQuantity,Monetary,AvgOrderValue,TotalReturns,ReturnRate,AvgQuantityPerInvoice,MaxQuantity,MinQuantity,StdQuantity,MeanPrice,MaxPrice,MinPrice,StdPrice,MostCommonHour,MostCommonWeekday,AvgDaysBetweenOrders
CustomerID,,,,,,,,,,,,,,,,,,,
12346,326,726,12,74285,77556.46,6463.038333,0,0.0,2184.852941,74215,1,12727.403892,6.100000,7.49,1.00,1.883066,13,0,12.0
12347,2,404,8,2967,4921.53,615.191250,0,0.0,13.364865,240,2,17.337408,2.498063,12.75,0.25,2.230495,14,1,1.0
12348,75,438,5,2714,2019.40,403.880000,0,0.0,53.215686,144,1,48.700848,3.786275,40.00,0.29,10.677136,14,0,7.0
12349,19,589,4,1624,4428.69,1107.172500,0,0.0,9.280000,48,1,7.770886,8.459657,300.00,0.42,32.808193,9,3,3.0
12350,310,310,1,197,334.40,334.400000,0,0.0,11.588235,24,1,4.345383,3.841176,40.00,0.85,9.334751,16,2,0.0
